In [1]:
import os
import librosa
import soundfile as sf
from scipy.signal import medfilt

In [2]:
def denoise(input_file, output_file):
    # 读取音频文件
    audio, sample_rate = librosa.load(input_file)
    # 对音频进行中值滤波
    denoised_audio = medfilt(audio, kernel_size=3)
    # 保存去噪后的音频文件
    sf.write(output_file, denoised_audio, sample_rate, sr=sample_rate)

In [ ]:
def process_folder(input_folder, output_folder):
    # 确保输出文件夹存在，没有的话自动创建
    os.makedirs(output_folder, exist_ok=True)
    # 处理输入文件夹中的所有 WAV 文件
    for file in os.listdir(input_folder):
        if file.endswith(".wav"):
            input_file_path = os.path.join(input_folder, file)
            output_file_path = os.path.join(output_folder, file)
            # 执行去噪函数
            denoise(input_file_path, output_file_path)
            print(f"{file} processed and saved.")
        else:
            print(f"{file} is not a WAV file.")
    print("All files processed.")

In [ ]:
# 替换为输入和输出文件夹路径
input_folder = r"E:\数据集\ShipEar\shipsEar_AUDIOS"
output_folder = r"E:\数据集\ShipEar\data_preprocessing\audio_noise_reduction"

In [ ]:
# 处理整个文件夹中的所有文件
process_folder(input_folder, output_folder)